# Time Separated Model
This origin comming from https://www.kaggle.com/code/lblhandsome/optiver-robust-best-single-model/notebook

In [1]:
from pathlib import Path
import os
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import gc  # Garbage collection for memory management
import os  # Operating system-related functions
import time  # Time-related functions
import warnings  # Handling warnings
from itertools import combinations  # For creating combinations of elements
from warnings import simplefilter  # Simplifying warning handling

# 📦 Importing machine learning libraries
import joblib  # For saving and loading models
import lightgbm as lgb  # LightGBM gradient boosting framework
import numpy as np  # Numerical operations
import pandas as pd  # Data manipulation and analysis
from sklearn.metrics import mean_absolute_error  # Metric for evaluation
from sklearn.model_selection import KFold, TimeSeriesSplit  # Cross-validation techniques

# 🤐 Disable warnings to keep the code clean
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

max_lookback = np.nan  # Maximum lookback (not specified)
split_day = 435  # Split day for time series data

warnings.filterwarnings("ignore")

PREV_MAX = 80
seed = 2023

# For kaggle environment
if os.environ.get("KAGGLE_DATA_PROXY_TOKEN") != None:
    BASE_OUTPUT_PATH = Path(f'/kaggle/working')
    BASE_INPUT_PATH = Path(f'/kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/test.csv')
    
    IS_LOCAL = False # If kaggle environment, set False
    IS_TRAIN = True # If kaggle environment, set True
    IS_INFER = True # If kaggle environment, set True

    if IS_LOCAL:
        SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/sample_submission.csv')
        REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/revealed_targets.csv')

    stopping_rounds = 30 # early_stopping用コールバック関数
    num_boost_round = 3000 # 計算回数
    num_folds = 5 # クロスバリデーションの分割数
    DEVICE = 'cpu' # cpu or gpu


    lgb_params = {
        'task': 'train',                   # 学習
        'boosting_type': 'gbdt',           # GBDT
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'learning_rate': 0.01,             # 学習率
        'lambda_l1': 0.5,                  # L1正則化項の係数
        'lambda_l2': 0.5,                  # L2正則化項の係数
        'num_leaves': 10,                  # 最大葉枚数
        'feature_fraction': 0.5,           # ランダムに抽出される列の割合
        'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
        "device": DEVICE
    }


    """
    lgb_params = {
        'task': 'train',                   # 学習
        'objective': 'regression',                # 目的関数の種類。ここでは回帰タスクを指定
        'metric': 'rmse',                          # 評価指標
        'boosting_type': 'gbdt',                  # ブースティングタイプ。勾配ブースティング決定木
        "n_estimators": 32,                        # ブースティングに使用する木の数。多いほど性能が向上するが計算コストが増加
        "num_leaves": 64,                         # 木に存在する最大の葉の数。大きい値は精度を向上させるが過学習のリスクが増加
        "subsample": 0.8,                         # 各木のトレーニングに使用されるデータの割合。過学習を防ぐために一部のデータをサンプリング
        "colsample_bytree": 0.8,                  # 木を構築する際に使用される特徴の割合。特徴のサブセットを使用し過学習を防ぐ
        "learning_rate": 0.01,                 # 学習率。小さい値は堅牢なモデルを生成するが収束に時間がかかる
        'max_depth': 32,                           # 木の最大の深さ。深い木は複雑なモデルを作成するが過学習のリスクがある
        "device": DEVICE,                         # トレーニングに使用するデバイス（CPUまたはGPU）
        "verbosity": -1,                          # LightGBMのログ出力のレベル。-1はログを出力しないことを意味する
        "importance_type": "gain",                # 特徴重要度を計算する際の指標。"gain"は分割による平均情報利得
        'lambda_l1': 0.5,                         # L1正則化項の係数。過学習を防ぐためにモデルの複雑さにペナルティを課す
        'lambda_l2': 0.5,                         # L2正則化項の係数。同じく過学習を防ぐ
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
    }
    """

# For local environment
else:
    BASE_OUTPUT_PATH = Path(f'../output')
    BASE_INPUT_PATH = Path(f'../kaggle/input/optiver-trading-at-the-close')
    TRAIN_FILE = Path(f'{BASE_INPUT_PATH}/train.csv')
    TEST_FILE = Path(f'{BASE_INPUT_PATH}/test.csv')

    SAMPLE_SUBMISSION_FILE = Path(f'{BASE_INPUT_PATH}/sample_submission.csv')
    REVEALED_TARGETS_FILE = Path(f'{BASE_INPUT_PATH}/revealed_targets.csv')
    IS_LOCAL = True
    IS_TRAIN = True
    IS_INFER = True
    TARGET_STOCK_IDS = [0,1,2,3,4,5,6,7,8,9]

    # For training
    stopping_rounds = 1 # early_stopping用コールバック関数
    num_boost_round = 1 # 計算回数
    num_folds = 5 # クロスバリデーションの分割数
    DEVICE = 'cpu' # cpu or gpu

    lgb_params = {
        'task': 'train',                   # 学習
        'boosting_type': 'gbdt',           # GBDT
        'objective': 'regression',         # 回帰
        'metric': 'rmse',                  # 損失（誤差）
        'learning_rate': 0.01,             # 学習率
        'lambda_l1': 0.5,                  # L1正則化項の係数
        'lambda_l2': 0.5,                  # L2正則化項の係数
        'num_leaves': 10,                  # 最大葉枚数
        'feature_fraction': 0.5,           # ランダムに抽出される列の割合
        'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
        'bagging_freq': 5,                 # バギング実施頻度
        'min_child_samples': 10,           # 葉に含まれる最小データ数
        'seed': seed,                       # シード値
        "device": DEVICE
    }


print(f"BASE_OUTPUT_PATH: {BASE_OUTPUT_PATH}")
print(f"BASE_INPUT_PATH: {BASE_INPUT_PATH}")
print(f"TRAIN_FILE: {TRAIN_FILE}")
print(f"TEST_FILE: {TEST_FILE}")
print(f"IS_LOCAL: {IS_LOCAL}")
print(f"IS_TRAIN: {IS_TRAIN}")
print(f"IS_INFER: {IS_INFER}")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


BASE_OUTPUT_PATH: /kaggle/working
BASE_INPUT_PATH: /kaggle/input/optiver-trading-at-the-close
TRAIN_FILE: /kaggle/input/optiver-trading-at-the-close/train.csv
TEST_FILE: /kaggle/input/optiver-trading-at-the-close/test.csv
IS_LOCAL: False
IS_TRAIN: True
IS_INFER: True


# Functions

In [2]:
from typing import Sequence, Tuple

import pandas as pd

# for local execution
class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = [TEST_FILE, REVEALED_TARGETS_FILE, SAMPLE_SUBMISSION_FILE]
        self.group_id_column: str = 'time_id'
        self.export_group_id_column: bool = True
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open('submission.csv', 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()

In [3]:
def pd_display_max():
    pd.set_option('display.max_rows', None)  # 行の最大表示数を無制限に設定
    pd.set_option('display.max_columns', None)  # 列の最大表示数を無制限に設定
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

def pd_clear_display_max():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 10)
    pd.set_option('display.width', None)  # 表示幅を拡張
    pd.set_option('display.max_colwidth', None)  # 列の幅を最大に設定

In [4]:
# 🧹 Function to reduce memory usage of a Pandas DataFrame
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    
    # 📏 Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    # 🔄 Iterate through each column in the DataFrame
    for col in df.columns:
        col_type = df[col].dtype

        # Check if the column's data type is not 'object' (i.e., numeric)
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # Check if the column's data type is an integer
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    # ℹ️ Provide memory optimization information if 'verbose' is True
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")

    # 🔄 Return the DataFrame with optimized memory usage
    return df

In [5]:
# 🏎️ Import Numba for just-in-time (JIT) compilation and parallel processing
from numba import njit, prange

# 📊 Function to compute triplet imbalance in parallel using Numba
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    # 🔁 Loop through all combinations of triplets
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        
        # 🔁 Loop through rows of the DataFrame
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            # 🚫 Prevent division by zero
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

# 📈 Function to calculate triplet imbalance for given price data and a DataFrame
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance using the Numba-optimized function
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [6]:
# 📊 Function to generate imbalance features
def imbalance_features(df):
    if DEVICE == 'gpu':
        import cudf
        df = cudf.from_pandas(df)
    
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    # V1 features
    # Calculate various features using Pandas eval function
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("ask_price + bid_price")/2
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("imbalance_size-matched_size")/df.eval("matched_size+imbalance_size")
    df["size_imbalance"] = df.eval("bid_size / ask_size")
    
    # Create features for pairwise price imbalances
    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
        
    # V2 features
    # Calculate additional features
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['match_balance'] = ( df['matched_size']  + (df['imbalance_buy_sell_flag'] * df['imbalance_size'])) / df['matched_size']
    
    # Calculate various statistical aggregation features
    
        
    # V3 features
    # Calculate shifted and return features for specific columns
    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
    
    # Calculate diff features for specific columns
    for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size']:
        for window in [1, 2, 3, 10]:
            df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)
    if DEVICE == 'gpu':
        df = df.to_pandas()
    # Replace infinite values with 0
    return df.replace([np.inf, -np.inf], 0)

def numba_imb_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    # Calculate triplet imbalance features using the Numba-optimized function
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values
    return df

# 📅 Function to generate time and stock-related features
def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    df["minute"] = df["seconds_in_bucket"] // 60  # Minutes

    # Map global features to the DataFrame
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df


def cal_vix(df, group_key, target_col, period):
    return df.groupby(group_key)[target_col].transform(lambda x: np.log(x).diff().rolling(period).std())


def generate_historical_features(df, is_train):
    for col in ['wap', 'match_balance']:
        for window in [1, 2, 3, 10]:
            col_name = f"{col}_diff_{window}"
            df[col_name] = df.groupby("stock_id")[col].diff(window)
            #df[col_name] = df[col_name].fillna(0)  # NaNを0で置き換える
        for period in [5]:
            col_name = f"{col}_vix_{period}"
            df[col_name] = cal_vix(df, ['stock_id', 'date_id'], col, period)

    df = df.replace([np.inf, -np.inf], 0)
    return df

# 🚀 Function to generate all features by combining imbalance and other features
def generate_all_features(df, is_train):
    prev_cols = list(df.columns)

    # Generate imbalance features
    df = imbalance_features(df)
    df = numba_imb_features(df)
    df = generate_historical_features(df, is_train)
    df = other_features(df)
    
    generated_feature_name = list(set(df.columns) - set(prev_cols))
    gc.collect()  # Perform garbage collection to free up memory

    return df, generated_feature_name

def normarized_features(df):
    df['normalized_wap'] = (df['wap'] - global_population_wap['mean']) / global_population_wap['std']
    df['normalized_match_balance'] = (df['match_balance'] - global_population_mathch_balance['mean']) / global_population_mathch_balance['std']
    return df

# Generationg train dataset

In [7]:
def load_train_dataset():
    df = pd.read_csv(TRAIN_FILE)
    # 🧹 Remove rows with missing values in the "target" column
    df = df.dropna(subset=["target"])
    # 🔁 Reset the index of the DataFrame and apply the changes in place
    df.reset_index(drop=True, inplace=True)
    return df


In [8]:
# Check if the code is running in offline or online mode
df_tmp_train = load_train_dataset()

if IS_LOCAL:
    # In local mode, stock id TARGET_STOCK_ID is used for training
    df_tmp_train = df_tmp_train[df_tmp_train["stock_id"].isin(TARGET_STOCK_IDS)]

# In online mode, use the entire dataset for training
df_train = df_tmp_train

# Display a message indicating online mode
print("Online mode")

if IS_TRAIN:
    global_stock_id_feats = {
        "median_size": df_train.groupby("stock_id")["bid_size"].median() + df_train.groupby("stock_id")["ask_size"].median(),
        "std_size": df_train.groupby("stock_id")["bid_size"].std() + df_train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": df_train.groupby("stock_id")["bid_size"].max() - df_train.groupby("stock_id")["bid_size"].min(),
        "median_price": df_train.groupby("stock_id")["bid_price"].median() + df_train.groupby("stock_id")["ask_price"].median(),
        "std_price": df_train.groupby("stock_id")["bid_price"].std() + df_train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": df_train.groupby("stock_id")["bid_price"].max() - df_train.groupby("stock_id")["ask_price"].min(),
    }

    df_train, generated_feature_name = generate_all_features(df_train, True)

    global_population_wap = df_train['wap'].describe()
    global_population_mathch_balance = df_train['match_balance'].describe()

    # normarize
    # df_train = normarized_features(df_train)

    print("Build Online Train Feats Finished.")

    df_train = reduce_mem_usage(df_train)

Online mode
Build Online Train Feats Finished.


In [9]:
# feature selection
feature_name = [
    "wap_vix_5", "reference_price_shift_10", "matched_size_ret_10",
    "matched_size_shift_10", "match_balance_vix_5", "ask_price_bid_price_reference_price_imb2",
    "seconds_in_bucket", "match_balance_diff_10", "imbalance_size_ret_10",
    "ask_size_diff_10", "imbalance_size_shift_10", "reference_price",
    "ask_price_bid_price_imb", "reference_price_ret_10", "all_sizes_mean",
    "matched_size", "bid_size_diff_10", "volume", "reference_price_shift_3",
    "bid_size", "bid_price_wap_reference_price_imb2", "ask_size",
    "ask_size_diff_3", "reference_price_bid_price_imb", "reference_price_wap_imb",
    "all_prices_kurt", "matched_size_bid_size_ask_size_imb2", "bid_price",
    "wap_diff_10", "bid_size_diff_3", "all_prices_std", "bid_size_ask_size_imbalance_size_imb2",
    "all_prices_skew", "all_sizes_skew", "bid_price_wap_imb",
    "ask_price_diff_10", "ask_price_wap_imb", "imbalance_size_ret_3",
    "matched_size_shift_3", "reference_price_shift_1", "all_prices_mean",
    "imbalance_size", "matched_size_ret_3", "reference_price_shift_2",
    "ask_price", "ask_size_diff_2", "bid_size_diff_2", "price_pressure",
    "reference_price_ask_price_imb", "matched_size_shift_1", "bid_size_diff_1",
    "market_urgency", "wap_diff_3", "price_spread", "all_sizes_std",
    "matched_size_shift_2", "imbalance_size_shift_3", "bid_price_diff_10",
    "ask_size_diff_1", "far_price", "reference_price_ret_3", "match_balance",
    "wap_diff_2", "wap_diff_1", "matched_imbalance", "wap"
]
feature_name

['wap_vix_5',
 'reference_price_shift_10',
 'matched_size_ret_10',
 'matched_size_shift_10',
 'match_balance_vix_5',
 'ask_price_bid_price_reference_price_imb2',
 'seconds_in_bucket',
 'match_balance_diff_10',
 'imbalance_size_ret_10',
 'ask_size_diff_10',
 'imbalance_size_shift_10',
 'reference_price',
 'ask_price_bid_price_imb',
 'reference_price_ret_10',
 'all_sizes_mean',
 'matched_size',
 'bid_size_diff_10',
 'volume',
 'reference_price_shift_3',
 'bid_size',
 'bid_price_wap_reference_price_imb2',
 'ask_size',
 'ask_size_diff_3',
 'reference_price_bid_price_imb',
 'reference_price_wap_imb',
 'all_prices_kurt',
 'matched_size_bid_size_ask_size_imb2',
 'bid_price',
 'wap_diff_10',
 'bid_size_diff_3',
 'all_prices_std',
 'bid_size_ask_size_imbalance_size_imb2',
 'all_prices_skew',
 'all_sizes_skew',
 'bid_price_wap_imb',
 'ask_price_diff_10',
 'ask_price_wap_imb',
 'imbalance_size_ret_3',
 'matched_size_shift_3',
 'reference_price_shift_1',
 'all_prices_mean',
 'imbalance_size',
 'ma

## global_stock_id_feats

In [10]:
df_global_stock_id_feats = pd.DataFrame(global_stock_id_feats)
df_global_stock_id_feats

,median_size,std_size,ptp_size,median_price,std_price,ptp_price
stock_id,,,,,,
0,42739.16,132986.920030,5898989.29,1.999695,0.003353,0.017414
1,25548.50,66444.908534,693898.57,1.999827,0.005588,0.029370
2,26228.10,75674.654248,1069837.58,2.000200,0.005333,0.051622
3,41667.00,93875.770520,1928848.21,1.999980,0.002903,0.018551
4,34014.58,80670.274550,1604065.54,1.999816,0.003717,0.017379
...,...,...,...,...,...,...
195,51941.55,98218.034986,2761658.98,1.999930,0.003051,0.014076
196,42476.95,78070.064244,459657.43,2.000042,0.003416,0.017398
197,30070.04,71964.174939,1575293.79,1.999984,0.004696,0.020387


## df_train_feats

In [11]:
df_train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,match_balance_vix_5,dow,seconds,minute,global_median_size,global_std_size,global_ptp_size,global_median_price,global_std_price,global_ptp_price
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,NaN,NaN,0.999812,...,NaN,0,0,0,42739.160156,132986.921875,5.898990e+06,1.999695,0.003353,0.017414
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,NaN,NaN,0.999896,...,NaN,0,0,0,25548.500000,66444.906250,6.938986e+05,1.999827,0.005588,0.029370
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,NaN,NaN,0.999403,...,NaN,0,0,0,26228.099609,75674.656250,1.069838e+06,2.000200,0.005333,0.051622
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,NaN,NaN,0.999999,...,NaN,0,0,0,41667.000000,93875.773438,1.928848e+06,1.999980,0.002903,0.018551
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,NaN,NaN,0.999394,...,NaN,0,0,0,34014.578125,80670.273438,1.604066e+06,1.999816,0.003717,0.017379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237887,195,480,540,2.440723e+06,-1,1.000317,28280362.00,0.999734,0.999734,1.000317,...,0.010796,0,0,9,51941.550781,98218.031250,2.761659e+06,1.999930,0.003051,0.014076
5237888,196,480,540,3.495105e+05,-1,1.000643,9187699.00,1.000129,1.000386,1.000643,...,0.016960,0,0,9,42476.949219,78070.062500,4.596574e+05,2.000042,0.003416,0.017398
5237889,197,480,540,0.000000e+00,0,0.995789,12725436.00,0.995789,0.995789,0.995789,...,0.032485,0,0,9,30070.039062,71964.171875,1.575294e+06,1.999984,0.004696,0.020387
5237890,198,480,540,1.000899e+06,1,0.999210,94773272.00,0.999210,0.999210,0.998970,...,0.004698,0,0,9,304739.250000,354682.781250,2.159163e+06,1.999917,0.003146,0.015738


# Model Training

In [12]:
# 📦 Import necessary libraries
import numpy as np
#import lightgbm as lgb
import optuna.integration.lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import gc
import os
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import lightgbm as lgb
import numpy as np
from dataclasses import dataclass

@dataclass
class Model:
    model: lgb.Booster
    fold: int
    feature_importance: pd.DataFrame
    score: float
    best_iteration: int
    train_time: float = None
    weight: float = None

def train_model(train_x, train_y, val_x, val_y):
    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y, reference=trains)

    verbose_eval = 0
    model = lgb.train(
        lgb_params,
        trains,
        valid_sets=[valids, trains], # 検証データ
        valid_names=['Train', 'Valid'],    # データセット名前
        num_boost_round=num_boost_round,
        callbacks=[
                lgb.early_stopping(stopping_rounds=stopping_rounds, verbose=True),
                lgb.log_evaluation(verbose_eval)
        ]
    )
    return model

def cross_train(df, key, n_splits, feature_name, valid_name):
    """ For Cross Train

    Args:
        df (_type_): _description_
        n_splits (_type_): _description_

    Returns:
        _type_: _description_
    """
    print(f"Cross Train key id {key}: start, shape: {df.shape}, n_splits: {n_splits}")

    models = []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    df.reset_index(drop=True, inplace=True)
    
    for fold, (train_indices, valid_indices) in enumerate(kf.split(df)):
        print(f"{key}: {fold} start")
        now_time = time.time()
        X_train, X_valid = df[feature_name].iloc[train_indices], df[feature_name].iloc[valid_indices]
        y_train, y_valid = df[valid_name].loc[train_indices], df[valid_name].loc[valid_indices]
        print(f"X_train: {X_train.shape}, X_valid: {X_valid.shape}, y_train: {y_train.shape}, y_valid: {y_valid.shape}")

        model = train_model(X_train, y_train, X_valid, y_valid)

        y_valid_pred = model.predict(X_valid)
        
        # For Debug
        #y_valid_df = pd.DataFrame(y_valid, columns=[valid_name])
        #y_valid_df["pred"] = y_valid_pred
        #y_valid_df.to_csv('filename.csv', index=False)

        score = mean_absolute_error(y_valid, y_valid_pred)
        train_time = time.time() - now_time
        m = Model(model, fold, model.feature_importance(), score, model.best_iteration, train_time, weight= 1 / n_splits)

        models.append(m)
        print(f"{key}: {fold} end, score: {score}, time: {train_time}")
        
        del X_train, X_valid, y_train, y_valid
        gc.collect()

    return key, models

#cross_train(x, x.name, num_folds, feature_name, "target")
models = df_train.groupby("seconds_in_bucket").apply(lambda x: cross_train(df=x, key=x.name, n_splits=num_folds, feature_name=feature_name, valid_name="target"))
# modelsを辞書に変換
models = {key: model for key, model in models}

Cross Train key id 0: start, shape: (95232, 127), n_splits: 5
0: 0 start
X_train: (76185, 66), X_valid: (19047, 66), y_train: (76185,), y_valid: (19047,)
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026826 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15548
[LightGBM] [Info] Number of data points in the train set: 76185, number of used features: 61
[LightGBM] [Info] Start training from score -0.060223
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[887]	Valid's rmse: 10.4197	Train's rmse: 10.7465
0: 0 end, score: 7.463119924797723, time: 4.893448829650879
0: 1 start
X_train: (76185, 66), X_valid: (19047, 66), y_train: (76185,), y_valid: (19047,)
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025957 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15549
[

In [13]:
# check model quality
data = []

for key, i_models in models.items():
    for model in i_models:
        score = model.score
        best_iteration = model.best_iteration
        fold = model.fold
        train_time = model.train_time
        data.append({"key": key, "fold": fold, "score": score, "best_iteration": best_iteration, "train_time": train_time})

df_model = pd.DataFrame(data)
df_model.describe()

,key,fold,score,best_iteration,train_time
count,275.000000,275.000000,275.000000,275.000000,275.000000
mean,270.000000,2.000000,6.270785,455.036364,3.146085
std,159.034496,1.416792,1.150232,178.329061,0.931877
min,0.000000,0.000000,4.846399,169.000000,1.713536
25%,130.000000,1.000000,5.594484,320.000000,2.424665
50%,270.000000,2.000000,5.958135,432.000000,3.017693
75%,410.000000,3.000000,6.311178,552.500000,3.648897
max,540.000000,4.000000,9.401089,1195.000000,6.254058


In [14]:
# Initialize an empty DataFrame for aggregated importances
aggregated_importance = pd.DataFrame(index=feature_name, columns=['importance'])

# Aggregate the importances from each model
for key, i_models in models.items():
    for model in i_models:
        importance = pd.DataFrame({'feature': feature_name, 'importance': model.feature_importance})
        aggregated_importance = aggregated_importance.add(importance.set_index('feature'), fill_value=0)


if models:
    aggregated_importance['importance'] /= len(df_model)

pd_display_max()
# Sort the features by importance
aggregated_importance = aggregated_importance.sort_values(by='importance', ascending=False)
aggregated_importance

,importance
market_urgency,285.923636
reference_price_wap_imb,194.276364
match_balance,135.992727
match_balance_diff_10,126.167273
wap_diff_3,115.392727
wap_diff_2,115.272727
bid_price_wap_imb,111.443636
wap_diff_1,109.621818
ask_price_wap_imb,107.941818
wap_vix_5,105.007273


In [15]:
pd_clear_display_max()

# Infer

In [16]:
# 📉 Define a function to adjust prices based on volumes
def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)  # 🧮 Calculate standard error based on volumes
    step = np.sum(prices) / np.sum(std_error)  # 🧮 Calculate the step size based on prices and standard error
    out = prices - std_error * step  # 💰 Adjust prices by subtracting the standardized step size
    return out

y_min, y_max = -64, 64

if IS_INFER:
    if IS_LOCAL:
        print("Infer Local")
        env = make_env()
    else:
        print("Infer Submission")
        import optiver2023
        env = optiver2023.make_env()
    iter_test = env.iter_test()

    #df_prediction = pd.DataFrame()
    #df_revealed_targets = pd.DataFrame()

    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        df_test, generated_feature_name = generate_all_features(test, False)
        df_test_grouped = df_test.groupby('seconds_in_bucket')

        for seconds_in_bucket, df in df_test_grouped:
            print(f"prdict: {test['date_id'][0]}, {seconds_in_bucket}")
            if seconds_in_bucket in models:
                target_models = models[seconds_in_bucket]
                df = df.sort_values(by=['stock_id', 'seconds_in_bucket', 'date_id']).reset_index(drop=True)

                predictions = np.mean([m.model.predict(df[feature_name]) for m in target_models], 0)
                zerosum_predictions = zero_sum(predictions, test['bid_size'] + test['ask_size'])
                clipped_predictions = np.clip(zerosum_predictions, y_min, y_max)  # 📏 Clip predictions within a specified range
                sample_prediction['target'] = clipped_predictions

                #df_prediction = pd.concat([df_prediction, sample_prediction])
                #df_revealed_targets = pd.concat([df_revealed_targets, revealed_targets])
            else:
                raise Exception(f"seconds_in_bucket {seconds_in_bucket} is not in models")
        env.predict(sample_prediction)


Infer Submission
This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
prdict: 478, 0
prdict: 478, 10
prdict: 478, 20
prdict: 478, 30
prdict: 478, 40
prdict: 478, 50
prdict: 478, 60
prdict: 478, 70
prdict: 478, 80
prdict: 478, 90
prdict: 478, 100
prdict: 478, 110
prdict: 478, 120
prdict: 478, 130
prdict: 478, 140
prdict: 478, 150
prdict: 478, 160
prdict: 478, 170
prdict: 478, 180
prdict: 478, 190
prdict: 478, 200
prdict: 478, 210
prdict: 478, 220
prdict: 478, 230
prdict: 478, 240
prdict: 478, 250
prdict: 478, 260
prdict: 478, 270
prdict: 478, 280
prdict: 478, 290
prdict: 478, 300
prdict: 478, 310
prdict: 478, 320
prdict: 478, 330
prdict: 478, 340
prdict: 478, 350
prdict: 478, 360
prdict: 478, 370
prdict: 478, 380
prdict: 478, 390
prdict: 478, 400
prdict: 478, 410
prdict: 478, 420
prdict: 478, 430
prdict: 478, 440
prdict: 478, 450
prdict: 478, 460
prdict: 478, 470
prdict: 478, 480
prdict: 478, 490
prdict: 478, 500
pr